In [9]:
print(robot.one_shot(text='你是谁？'))

我是一个智能对话机器人，可以回答你提出的问题并与你进行交流。有什么我可以帮助你的吗？


In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

/Users/youniverse/miniconda3/envs/comment_analysis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files:  60%|██████    | 18/30 [08:31<05:41, 28.43s/it]


In [3]:
import pandas as pd


df = pd.read_csv('./buffer/comment_samples.csv')
print(len(df))
print(df.columns)

191
Index(['ASIN', '标题', '标题(翻译)', '内容', '内容(翻译)', 'VP评论', 'Vine Voice评论', '型号',
       '星级', '好评点1', '好评点2', '好评点3', '好评点4', '好评点5', '差评点1', '差评点2', '差评点3',
       '差评点4', '差评点5', '用户期望', '备注', '赞同数', '图片数量', '图片地址', '是否有视频', '视频地址',
       '评论链接', '评论人', '头像地址', '所属国家', '评论人主页', '红人计划链接', '评论时间'],
      dtype='object')


In [6]:

length = len(df)
for i in range(4):
    comment = df.iloc[i]['内容']
    print(comment)

Wonderful desk! Wonderful people to work with. Thank you
It’s a nice desk was pretty easy to assemble
The desk is beautiful looking although it is a little bit deceiving. Kinda wish I ordered a different one. I agree with other reviews that the set up takes forever and there are SO MANY PARTS! I am petite/5’3 and bought a computer chair that is equivalent to the height of a dining room table chair, nothing special and I have a hard time putting my legs under so I do wish it was taller. Also, the drawers aren’t as deep as the desk is so that’s a bummer they are so small. It’s just big enough to fit a 8 x 10 sheet of paper. Overall looks beautiful but functionality can be improved.
I just received this desk. The box was badly damaged and most of the pieces inside are chipped and even some the wood has split. Don't buy this desk.


In [23]:
import yaml
from modules.agent import CommentAnalysisAgent


with open("openai_keys.yaml", "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)

my_key = data['tom']['key']
robot = CommentAnalysisAgent(openai_key=my_key, model="gpt-4o-mini-2024-07-18")  # gpt-4o-2024-08-06  gpt-4o-mini-2024-07-18

prompt = '''你是一个评论分析师。现在有个任务是根据买家的评论，提取出里面的好评点和差评点。你需要按照评论提及的点的顺序，依次输出是好评还是差评+评论角度；输出格式为 {'好评点1': 'xxx', ..., '好评点n': 'xxx'}，如果没有信息点则输出{}
评论的角度有：<安装难度>，<安装说明书>，<安装时长>，<搬运难度>，<包装保护性>，<抽屉承重能力>，<抽屉滑轨顺畅度>，<抽屉收纳空间>，<错件情况>，<附加电源/USB 插座>，<柜门开合顺畅度>，<客服质量>，<螺丝&孔位匹配度>，<清洁方便性>，<气味情况>，<缺件情况>，<商品与详情页匹配度>，<使用噪音>，<腿部空间大小>，<五金件损坏情况>，<物流速度>，<线缆管理功能>，<桌板表面完整性>，<桌板材质>，<桌板厚度>，<桌面承重能力>，<桌面防水性>，<桌体破损情况>，<桌腿材质>，<桌腿防滑性>，<稳定性>，<外观>，<整装拆装>
Example1: comment='Wonderful desk! Wonderful people to work with. Thank you'; output={}
Example2: comment='It’s a nice desk was pretty easy to assemble'; output={'好评点1': '安装难度'}
Example3: comment='The desk is beautiful looking although it is a little bit deceiving. Kinda wish I ordered a different one. I agree with other reviews that the set up takes forever and there are SO MANY PARTS! I am petite/5’3 and bought a computer chair that is equivalent to the height of a dining room table chair, nothing special and I have a hard time putting my legs under so I do wish it was taller. Also, the drawers aren’t as deep as the desk is so that’s a bummer they are so small. It’s just big enough to fit a 8 x 10 sheet of paper. Overall looks beautiful but functionality can be improved.'; output={'好评点1': '外观', '差评点1': '腿部空间大小', '差评点2': '安装时长', '差评点3': '抽屉收纳空间'}
Now do: comment='''

ans = {}
for i in range(4, 15):
    comment = df.iloc[i]['内容']
    print(comment)
    task_text = prompt + "'" + comment + "'"
    rep = robot.one_shot(text=task_text)
    print(rep)
    
    ans[comment] = rep


The reason why I give 3 stars, is because the assembly was a pain. To long to finish it. Nothing is build, not even the drawer.
{'差评点1': '安装难度', '差评点2': '安装时长'}
It’s a pain in the ass to assemble but it’s amazing. The drawers glide perfectly and the feel of the material is amazing! I had to screw holes to drop cords but totally worth it.
{'差评点1': '安装难度', '好评点1': '抽屉滑轨顺畅度', '好评点2': '桌板材质'}
Complete trash. No file space. It’s supposed to be an executive desk and you cannot put files in it at all. False advertising and a waste of my damn money. I want a refund!
{'差评点1': '抽屉收纳空间', '差评点2': '商品与详情页匹配度'}
The desk is solid and fairly straightforward. Good storage space and true to color depicted on Amazon. Just one issue but for us it is pretty significant: The drawers are not very deep/long. They measure just 11-1/2” in length and 10” wide. The file drawer (lower right) is mainly where this is a problem for us. A typical file folder system has to be compacted to fit and forget about a hanging

In [ ]:
print(robot.one_shot(text=task_text))